# PIR - LSTM n°3 Hyperparameters Optimization

This notebook is used for hyperparameter optimization for the model implemented in LSTMn°3.ipynb

We aim to optimize the LSTM model's hyperparameters by using data up to June 2022 for training and validation, and the remaining part of 2022 as the test set.

## 1: Imports

In [ ]:
# Custom utility functions used in the project
import utils

# Disable GPU to avoid TensorFlow conflicts
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error

2026-01-10 19:19:43.718130: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-10 19:19:43.758821: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-10 19:19:46.563444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


The hyperparameters optimization is performed using Optuna's TPESampler, which provides an efficient and scalable alternative to grid and random search.

In [2]:
# imports
import optuna
import tqdm
from optuna.samplers import TPESampler

/home/pedoussa/.conda/envs/tf311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## 2: Hyperparameters Optimization

In [3]:
# Load the data
x = pd.read_csv('train_f_x.csv')
y = pd.read_csv('y_train_sncf.csv')

In [ ]:
# Data preparation
df_per_station_train, df_per_station_test = utils.prepare_train_data(x, y)

/home/pedoussa/Bureau/PIR 5A/utils.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['station_id'] = data['station'].map(station_mapping)
/home/pedoussa/Bureau/PIR 5A/utils.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['station_id'] = data['station'].map(station_mapping)


Hyperparameter tuning is conducted on a subset of 30 stations (out of 439) to significantly reduce runtime, under the assumption that optimal hyperparameters generalize across stations.

In [ ]:
# Sampling
sample_size = 30
seed = 50
stations_sample = utils.sample_stations(df_per_station_train, n=sample_size, seed=seed)

### 2.1: First attempt

The first optimization attempt included the choice of the activation function between 'tanh' and 'relu'. However, the use of ReLU activation later led to gradient explosion issues, which motivated us to restrict the activation function to 'tanh' in subsequent experiment.

In [ ]:
def objective(trial):
    # Hyperparameters Grid
    seq_len = trial.suggest_int("seq_len", 30, 120, step=15)
    units = trial.suggest_int("units", 20, 128)
    lr = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    activation = trial.suggest_categorical("activation", ["tanh", "relu"])
    
    total_error = 0 # Initialization

    for name_station in stations_sample:
        try:
            df_train = df_per_station_train[name_station]
            df_test = df_per_station_test[name_station]

            X_train = df_train[['job', 'ferie', 'vacances']]
            y_train = df_train['y']
            X_test = df_test[['job', 'ferie', 'vacances']]
            y_test = df_test['y']

            # Scaling
            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
            X_test_scaled = scaler_X.transform(X_test)

            # Training sequences
            X_train_seq, y_train_seq = utils.create_sequences_random(
                pd.DataFrame(X_train_scaled),
                pd.DataFrame(y_train_scaled),
                seq_len
            )

            # Test_sequences
            X_test_full = np.vstack([X_train_scaled[-seq_len:], X_test_scaled])
            X_test_seq = np.array([
                X_test_full[i:i+seq_len]
                for i in range(len(X_test))
            ]) # Parenthèse fermante ajoutée ici

            # Model definition
            model = Sequential([
                LSTM(units=units, activation=activation, input_shape=(seq_len, X_train.shape[1])),
                Dense(1)
            ])
            model.compile(optimizer=Adam(learning_rate=lr), loss='mse')

            # Short training for hyperparameter optimization
            model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=batch_size, verbose=0)

            # Prediction and error computation
            y_pred_scaled = model.predict(X_test_seq, verbose=0)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            
            rmse = np.sqrt(mean_absolute_percentage_error(y_test, y_pred))
            total_error += rmse
            
        except Exception as e:
            print(f"Error for station {name_station}: {e}")
            continue

    return total_error / len(stations_sample)

[I 2026-01-10 19:24:05,580] A new study created in memory with name: no-name-02f302f7-d30f-42f2-b4f7-f0054dab2895
  0%|          | 0/30 [00:00<?, ?it/s]/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass 

[I 2026-01-10 19:29:11,972] Trial 0 finished with value: 0.9421255748370522 and parameters: {'seq_len': 60, 'units': 123, 'learning_rate': 0.0029106359131330704, 'batch_size': 16, 'activation': 'relu'}. Best is trial 0 with value: 0.9421255748370522.


/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass 

...

[I 2026-01-10 20:47:05,632] Trial 27 finished with value: 0.9229687447310729 and parameters: {'seq_len': 120, 'units': 28, 'learning_rate': 0.002527066432305605, 'batch_size': 32, 'activation': 'tanh'}. Best is trial 24 with value: 0.7504421447393268.


/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass 

[I 2026-01-10 20:50:17,634] Trial 28 finished with value: 0.9482492738799326 and parameters: {'seq_len': 105, 'units': 90, 'learning_rate': 0.005728484654098207, 'batch_size': 64, 'activation': 'tanh'}. Best is trial 24 with value: 0.7504421447393268.


/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/pedoussa/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass 

[I 2026-01-10 20:55:37,401] Trial 29 finished with value: 0.9671746971594827 and parameters: {'seq_len': 60, 'units': 125, 'learning_rate': 0.001216147966541517, 'batch_size': 16, 'activation': 'tanh'}. Best is trial 24 with value: 0.7504421447393268.
Meilleurs paramètres : {'seq_len': 120, 'units': 32, 'learning_rate': 0.009866823292399145, 'batch_size': 32, 'activation': 'relu'}


In [ ]:
# Execution
run = 0
if run == 1:
    study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
    study.optimize(objective, n_trials=30, show_progress_bar=True)
else:
    print("run == 0")

In [ ]:
print("Best hyperparameters :", study.best_params)

Meilleurs paramètres : {'seq_len': 120, 'units': 32, 'learning_rate': 0.009866823292399145, 'batch_size': 32, 'activation': 'relu'}


### 2.2: Second attempt

Second hyperparameter optimization attempt unsing only the tanh activation function.

In [ ]:
def objective(trial):
    # Hyperparameters Grid
    seq_len = trial.suggest_int("seq_len", 70, 120, step=15)
    units = trial.suggest_int("units", 20, 128)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    total_error = 0 # Initialization

    for name_station in stations_sample:
        try:
            df_train = df_per_station_train[name_station]
            df_test = df_per_station_test[name_station]

            X_train = df_train[['job', 'ferie', 'vacances']]
            y_train = df_train['y']
            X_test = df_test[['job', 'ferie', 'vacances']]
            y_test = df_test['y']

            # Scaling
            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
            X_test_scaled = scaler_X.transform(X_test)

            # Training sequences
            X_train_seq, y_train_seq = utils.create_sequences_random(
                pd.DataFrame(X_train_scaled),
                pd.DataFrame(y_train_scaled),
                seq_len
            )

            # Test sequences
            X_test_full = np.vstack([X_train_scaled[-seq_len:], X_test_scaled])
            X_test_seq = np.array([
                X_test_full[i:i+seq_len]
                for i in range(len(X_test))
            ])

            # Model definition
            model = Sequential([
                Input(shape=(seq_len, X_train.shape[1])),
                LSTM(units=units, activation='tanh'),
                Dense(1)
            ])
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

            # Short training for hyperparameter optimization
            model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=batch_size, verbose=0)

            # Prediction and error computation
            y_pred_scaled = model.predict(X_test_seq, verbose=0)
            y_pred = scaler_y.inverse_transform(y_pred_scaled)
            
            rmse = np.sqrt(mean_absolute_percentage_error(y_test, y_pred))
            total_error += rmse
            
        except Exception as e:
            print(f"Error for station {name_station}: {e}")
            continue

    return total_error / len(stations_sample)

[I 2026-01-10 21:36:57,874] A new study created in memory with name: no-name-ce056424-157f-4764-ae12-1d9b6c954f2b
  0%|          | 0/20 [00:00<?, ?it/s]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(
Best trial: 0. Best value: 0.931501:   5%|▌         | 1/20 [07:00<2:13:03, 420.20s/it]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(


[I 2026-01-10 21:43:58,071] Trial 0 finished with value: 0.9315012836572233 and parameters: {'seq_len': 85, 'units': 123, 'learning_rate': 0.0029106359131330704, 'batch_size': 16}. Best is trial 0 with value: 0.9315012836572233.


Best trial: 0. Best value: 0.931501:  10%|█         | 2/20 [10:06<1:24:51, 282.88s/it]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(


[I 2026-01-10 21:47:04,823] Trial 1 finished with value: 0.9615602158418163 and parameters: {'seq_len': 70, 'units': 114, 'learning_rate': 0.0015930522616241021, 'batch_size': 64}. Best is trial 0 with value: 0.9315012836572233.


Best trial: 0. Best value: 0.931501:  15%|█▌        | 3/20 [12:34<1:02:35, 220.91s/it]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(


...

[I 2026-01-10 22:14:13,664] Trial 9 finished with value: 1.2189532518073742 and parameters: {'seq_len': 100, 'units': 120, 'learning_rate': 0.00015030900645056822, 'batch_size': 64}. Best is trial 3 with value: 0.9074125672829692.


Best trial: 3. Best value: 0.907413:  55%|█████▌    | 11/20 [40:09<31:25, 209.53s/it]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(


[I 2026-01-10 22:17:07,552] Trial 10 finished with value: 1.0629371098650802 and parameters: {'seq_len': 115, 'units': 69, 'learning_rate': 0.000724359894840497, 'batch_size': 64}. Best is trial 3 with value: 0.9074125672829692.


Best trial: 11. Best value: 0.869105:  60%|██████    | 12/20 [45:20<32:03, 240.46s/it]/home/pedoussa/.local/lib/python3.11/site-packages/optuna/distributions.py:684: UserWarning: The distribution is specified by [70, 120] and step=15, but the range is not divisible by `step`. It will be replaced with [70, 115].
  warnings.warn(


[I 2026-01-10 22:22:18,760] Trial 11 finished with value: 0.8691048082951898 and parameters: {'seq_len': 85, 'units': 76, 'learning_rate': 0.004546404153956352, 'batch_size': 16}. Best is trial 11 with value: 0.8691048082951898.


In [ ]:
# Execution
run = 0
if run == 1:
    study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=42))
    study.optimize(objective, n_trials=30, show_progress_bar=True)
else:
    print("run == 0")

In [ ]:
print("Best hyperparameters :", study.best_params)